In [24]:
# Welcome to the Kalshi REST v2 Starter Code!

# pypi client: recommended for more advanced programmers
#import kalshi_python

# starter client: recommended for all levels of programming experience (what this client is implemented using)
from KalshiClientsBaseV2ApiKey import ExchangeClient
import time
import json
import uuid

In [3]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,  # or provide a password if your key is encrypted
            backend=default_backend()
        )
    return private_key

In [ ]:
# To start off, you need to have created an account at https://kalshi.com (Production)
# or an account on the Demo https://demo.kalshi.co/

## if wanting to test in prod change this to prod
env = "demo"

if env == "prod":
    key_id = "" # change these to be your personal credentials
    private_key = load_private_key_from_file('kalshi-prod-key.key')
    api_base = "https://api.elections.kalshi.com/trade-api/v2"
else:
    key_id = "" # change these to be your personal credentials
    private_key = load_private_key_from_file('kalshi-demo-key.key')
    api_base = "https://demo-api.kalshi.co/trade-api/v2"

exchange_client = ExchangeClient(
    exchange_api_base = api_base,
    key_id = key_id,
    private_key = private_key,
)

# first we will check on the exchange status to confirm you are properly connected...
print(exchange_client.get_exchange_status())

{'exchange_active': True, 'trading_active': True}


In [7]:
# You can discover markets through the get_markets endpoint...

# and use query parameters to filter your search!
market_params = {
    'limit':100,
    'cursor':None, # passing in the cursor from the previous get_markets call
    'event_ticker': None,
    'series_ticker':None,
    'max_close_ts':None, # pass in unix_ts
    'min_close_ts':None, # pass in unix_ts
    'status':None,
    'tickers':None
}
markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in payload:', markets_response['markets'][0])
print()
print('cursor:', cursor)

keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in payload: {'ticker': 'KXQUICKSETTLE-25MAY23H1600-3', 'event_ticker': 'KXQUICKSETTLE-25MAY23H1600', 'market_type': 'binary', 'title': 'Will 1+1 equal 3 on May 23 at 16:00?', 'subtitle': '', 'yes_sub_title': '3', 'no_sub_title': '3', 'open_time': '2025-05-23T19:50:00Z', 'close_time': '2025-05-23T20:00:00Z', 'expected_expiration_time': '2025-05-23T20:00:00Z', 'expiration_time': '2025-05-23T20:00:00Z', 'latest_expiration_time': '2025-05-23T20:00:00Z', 'settlement_timer_seconds': 60, 'status': 'initialized', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 0, 'no_bid': 100, 'no_ask': 100, 'last_price': 0, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 0, 'volume_24h': 0, 'liquidity': 0, 'open_interest': 0, 'result': '', 'can_close_early': True, 'expiration_value': '', 'category': '', 'risk_limit_cents': 0, 'rules_primary': 'If 1+1 eq

In [8]:
# What are cursors and how do they work?
    
# The Cursor represents a pointer to the next page of records in the pagination.
# So this optional parameter, when filled, should be filled with the cursor string returned in a previous request to this end-point.
# Filling this would basically tell the api to get the next page containing the number of records passed on the limit parameter.
# On the other side not filling it tells the api you want to get the first page for another query.
# The cursor does not store any filters, so if any filter parameters like tickers, max_ts or min_ts were passed in the original query they must be passed again.

# Let's try it in action! Suppose we wanted to get the next 100 market objects...

market_params = {'limit':100,
                    'cursor':cursor, # passing in the cursor from the previous get_markets call
                    'event_ticker': None,
                    'series_ticker': None,
                    'max_close_ts': None, # pass in unix_ts
                    'min_close_ts': None, # pass in unix_ts
                    'status': None,
                    'tickers':None}

markets_response = exchange_client.get_markets(**market_params)
cursor = markets_response['cursor']

print('keys:', markets_response.keys())
print()
print('number of objects:', len(markets_response['markets'])) # 100 objects!
print()
print('first market in market_response payload:', markets_response['markets'][0]) # new markets!
print()
print('new cursor!', cursor)

keys: dict_keys(['markets', 'cursor'])

number of objects: 100

first market in market_response payload: {'ticker': 'KXSPOTIFYAUSTRALIAD-25MAY24-THA', 'event_ticker': 'KXSPOTIFYAUSTRALIAD-25MAY24', 'market_type': 'binary', 'title': 'Top Song on Daily Top Songs Australia on May 24, 2025?', 'subtitle': ':: Gracie Abrams', 'yes_sub_title': 'That’s So True', 'no_sub_title': 'That’s So True', 'open_time': '2025-05-23T21:00:00Z', 'close_time': '2025-05-25T03:59:00Z', 'expected_expiration_time': '2025-06-07T14:00:00Z', 'expiration_time': '2025-06-07T14:00:00Z', 'latest_expiration_time': '2025-06-07T14:00:00Z', 'settlement_timer_seconds': 300, 'status': 'initialized', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 0, 'no_bid': 100, 'no_ask': 100, 'last_price': 0, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 0, 'volume_24h': 0, 'liquidity': 0, 'open_interest': 0, 'result': '', 'can_close_early': True, 'expirati

In [9]:
# Next, let's look at event level data by passing an event ticker to the get_event endpoint...

event_ticker = markets_response['markets'][5]['event_ticker']
event_params = {'event_ticker': event_ticker}
event_response = exchange_client.get_event(**event_params)

print('keys:', event_response.keys())
print()
print('event object:', event_response['event'])
print()
print('first market in event_response payload:', event_response['markets'][0])

keys: dict_keys(['event', 'markets'])

event object: {'event_ticker': 'KXSPOTIFYAUSTRALIAD-25MAY24', 'series_ticker': 'KXSPOTIFYAUSTRALIAD', 'sub_title': 'On May 24, 2025 Chart', 'title': 'Top Song on Daily Top Songs Australia tomorrow?', 'collateral_return_type': 'MECNET', 'mutually_exclusive': True, 'category': 'Entertainment'}

first market in event_response payload: {'ticker': 'KXSPOTIFYAUSTRALIAD-25MAY24-ORD', 'event_ticker': 'KXSPOTIFYAUSTRALIAD-25MAY24', 'market_type': 'binary', 'title': '', 'subtitle': ':: Alex Warren', 'yes_sub_title': 'Ordinary', 'no_sub_title': 'Ordinary', 'open_time': '2025-05-23T21:00:00Z', 'close_time': '2025-05-25T03:59:00Z', 'expected_expiration_time': '2025-06-07T14:00:00Z', 'expiration_time': '2025-06-07T14:00:00Z', 'latest_expiration_time': '2025-06-07T14:00:00Z', 'settlement_timer_seconds': 300, 'status': 'initialized', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 0, 'no_bid': 100, 'no_ask': 100

In [10]:
# Next, let's look at series level data by passing a series ticker to the get_series endpoint! 
series_ticker = event_response['event']['series_ticker']
series_params = {'series_ticker': series_ticker}
series_response = exchange_client.get_series(**series_params)

print('keys:', series_response.keys())
print()
print('series object:', series_response['series'])
print()

keys: dict_keys(['series'])

series object: {'ticker': 'KXSPOTIFYAUSTRALIAD', 'frequency': 'daily', 'title': 'Daily Australia Spotify songs', 'category': 'Entertainment', 'tags': ['Music', 'Music charts'], 'settlement_sources': [{'url': 'https://charts.spotify.com/home', 'name': 'Spotify Charts'}], 'contract_url': 'https://kalshi-public-docs.s3.us-east-1.amazonaws.com/regulatory/product-certifications/SPOTIFYCHART.pdf'}



In [26]:
# Next let's look at the recent market history for a market
series_ticker = 'KXLLM1'
ticker =        'KXLLM1-25MAY31-GOOG'

market_history_params = {
    'series_ticker':    series_ticker,
    'market_ticker':    ticker,
    'period_interval':  1, # 1 minute candlesticks
    'start_ts':         int(time.time()) - 60 * 60 * 24 * 1, # passing a recent unix_ts
    'end_ts':           int(time.time()),         # passing a recent unix_ts
}
market_history_response = exchange_client.get_market_history(**market_history_params)

print(market_history_response)

print('keys:', market_history_response.keys())
print()
print('most recent market history object:', market_history_response['candlesticks'][-1])
print()

# and then also look at the most current view of the orderbook
market_history_params = {
    'ticker': ticker,
    'depth':  30
}
orderbook_response = exchange_client.get_orderbook(**market_history_params)

print('keys:', orderbook_response.keys())
print()
print('orderbook object:', orderbook_response)
print()

{'ticker': 'KXLLM1-25MAY31-GOOG', 'candlesticks': [{'end_period_ts': 1747951800, 'yes_bid': {'open': 0, 'low': 0, 'high': 0, 'close': 0}, 'yes_ask': {'open': 10, 'low': 10, 'high': 100, 'close': 100}, 'price': {'open': None, 'low': None, 'high': None, 'close': None, 'mean': None, 'previous': None}, 'volume': 0, 'open_interest': 0}]}
keys: dict_keys(['ticker', 'candlesticks'])

most recent market history object: {'end_period_ts': 1747951800, 'yes_bid': {'open': 0, 'low': 0, 'high': 0, 'close': 0}, 'yes_ask': {'open': 10, 'low': 10, 'high': 100, 'close': 100}, 'price': {'open': None, 'low': None, 'high': None, 'close': None, 'mean': None, 'previous': None}, 'volume': 0, 'open_interest': 0}

keys: dict_keys(['orderbook'])

orderbook object: {'orderbook': {'yes': None, 'no': None}}



In [28]:
# Now let's suppose we wanted to place a trade on one of these markets...
# to do so, we would first want to check out available balance...

current_balance = exchange_client.get_balance()
current_balance

{'balance': 385003}

In [ ]:
# Now that you have some balance, you might want to see how your current positions are doing...

positions_params = {
    'limit': None,
    'cursor': None,
    'settlement_status': None,
    'ticker': None,
    'event_ticker': None,
}

current_position = exchange_client.get_positions(**positions_params)
current_position

{'cursor': '',
 'market_positions': [{'ticker': 'BTCMAXY-24DEC31-90000',
   'total_traded': 5100,
   'position': 100,
   'market_exposure': 5100,
   'realized_pnl': 0,
   'resting_orders_count': 0,
   'fees_paid': 175,
   'last_updated_ts': '2024-06-03T18:04:41.052476Z'},
  {'ticker': 'BTCMAXY-24DEC31-80000',
   'total_traded': 9450,
   'position': 150,
   'market_exposure': 9450,
   'realized_pnl': 0,
   'resting_orders_count': 0,
   'fees_paid': 80,
   'last_updated_ts': '2024-07-31T20:13:10.833106Z'},
  {'ticker': 'LLM1-24DEC31-OAI',
   'total_traded': 1206,
   'position': 21,
   'market_exposure': 1206,
   'realized_pnl': 0,
   'resting_orders_count': 0,
   'fees_paid': 17,
   'last_updated_ts': '2024-06-20T22:39:35.771596Z'},
  {'ticker': 'LLM1-24DEC31-A',
   'total_traded': 1950,
   'position': 50,
   'market_exposure': 1950,
   'realized_pnl': 0,
   'resting_orders_count': 0,
   'fees_paid': 84,
   'last_updated_ts': '2024-07-29T15:07:29.283082Z'},
  {'ticker': 'CPICORE-24JUN-T0

In [29]:
# seems like some of your recent orders had been filled. To check on those we use the get_positions endpoint ...

fills_params = {
    'ticker':   None,
    'order_id': None,
    'min_ts':   None,
    'max_ts':   None,
    'limit':    None,
    'cursor':   None,
}

fills = exchange_client.get_fills(**fills_params)
fills

{'fills': [{'trade_id': '05c03cc5-7fcd-4158-8415-f59fa6238101',
   'ticker': 'KXBTCMAXY-25-DEC31-124999.99',
   'order_id': 'a630bbae-7596-4ce0-8645-37ae719cd3c0',
   'side': 'yes',
   'action': 'sell',
   'count': 2,
   'yes_price': 65,
   'no_price': 35,
   'is_taker': True,
   'created_time': '2025-05-22T12:12:03.313608Z'},
  {'trade_id': '3d082b2f-31a2-490a-ab10-1a0f615687b6',
   'ticker': 'KXBTCMAXY-25-DEC31-124999.99',
   'order_id': '8596a175-c94c-42d6-9fde-64f9b9463cf4',
   'side': 'no',
   'action': 'buy',
   'count': 2,
   'yes_price': 60,
   'no_price': 40,
   'is_taker': True,
   'created_time': '2025-05-22T12:10:35.472077Z'},
  {'trade_id': '06f1ced2-b5ec-4046-9327-ad6a4b7c6619',
   'ticker': 'KXDOGEMAX1-25-JUN01-0.99999999',
   'order_id': '555d4554-8263-4b80-a2ba-9ea3d70e737b',
   'side': 'no',
   'action': 'buy',
   'count': 1,
   'yes_price': 12,
   'no_price': 88,
   'is_taker': True,
   'created_time': '2025-05-21T00:05:43.087383Z'},
  {'trade_id': 'bc23c077-4dc7-489

In [30]:
# you may even want to check on some of your recent positions settled...

settlement_params = {
    'limit':  None,
    'cursor': None,
}

settlements = exchange_client.get_portfolio_settlements(**settlement_params)
settlements

{'settlements': [{'ticker': 'KXTRUMPPARDON-25MAY01-EA',
   'market_result': 'no',
   'yes_count': 1,
   'yes_total_cost': 30,
   'no_count': 5,
   'no_total_cost': 365,
   'revenue': 400,
   'settled_time': '2025-05-09T18:42:36.614628Z'},
  {'ticker': 'KXTRUMPPARDON-25MAY01-SB',
   'market_result': 'no',
   'yes_count': 0,
   'yes_total_cost': 0,
   'no_count': 5,
   'no_total_cost': 300,
   'revenue': 500,
   'settled_time': '2025-05-09T18:42:35.684066Z'},
  {'ticker': 'DEBATES24-3',
   'market_result': 'no',
   'yes_count': 0,
   'yes_total_cost': 0,
   'no_count': 2,
   'no_total_cost': 60,
   'revenue': 200,
   'settled_time': '2025-03-14T10:27:18.006217Z'},
  {'ticker': 'KXGRAMROTY-67-NLU',
   'market_result': 'yes',
   'yes_count': 50,
   'yes_total_cost': 3800,
   'no_count': 0,
   'no_total_cost': 0,
   'revenue': 5000,
   'settled_time': '2025-02-07T21:46:55.801047Z'},
  {'ticker': 'KXGRAMROTY-67-E',
   'market_result': 'no',
   'yes_count': 0,
   'yes_total_cost': 0,
   'no_c

In [31]:
# Now onto placing an order...
# There are many different ways to think about placing orders at Kalshi.
# The following param examples will walk through some of those

# Limit buy order for 10 units at 30c No on GDPW-22-A3

ticker = 'TESTING-5'

order_params = {
    'ticker':               ticker,
    'client_order_id':      str(uuid.uuid4()),
    'type':                 'limit',
    'action':               'buy',
    'side':                 'no',
    'count':                10,
    'yes_price':            None, # yes_price = 100 - no_price
    'no_price':             30,   # no_price =  100 - yes_price
    'expiration_ts':        None,
    'sell_position_floor':  None,
    'buy_max_cost':         None,
}

exchange_client.create_order(**order_params)

# EQUIVALENTLY, because buying No is equivalent to selling yes...

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'limit',
#                     'action':'sell',
#                     'side':'yes',
#                     'count':10,
#                     'yes_price':None, # yes_price = 100 - no_price
#                     'no_price':30, # no_price = 100 - yes_price
#                     'expiration_ts':None,
#                     'sell_position_floor':None,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)


# # Market sell order for 12 units Yes on GDPW-22-A3, without flipping position

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'market',
#                     'action':'sell',
#                     'side':'yes',
#                     'count':12,
#                     'yes_price':1,
#                     'no_price':None,
#                     'expiration_ts':None,
#                     'sell_position_floor':0,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)

# # EQUIVALENTLY, because buying No is equivalent to selling yes...

# order_params = {'ticker':ticker,
#                     'client_order_id':str(uuid.uuid4()),
#                     'type':'market',
#                     'action':'buy',
#                     'side':'no',
#                     'count':12,
#                     'yes_price':1,
#                     'no_price':None,
#                     'expiration_ts':None,
#                     'sell_position_floor':0,
#                     'buy_max_cost':None}

# exchange_client.create_order(**order_params)


{'ticker': 'TESTING-5', 'client_order_id': '5f82e4e5-ffa3-421a-a112-cd4b888cddb3', 'side': 'no', 'action': 'buy', 'count': 10, 'type': 'limit', 'no_price': 30}


{'order': {'order_id': '1c995370-d0a4-445d-94a9-be603645b082',
  'user_id': '5b044122-9dd6-4519-bc64-82f6872352d1',
  'ticker': 'TESTING-5',
  'status': 'resting',
  'yes_price': 70,
  'no_price': 30,
  'created_time': '2025-05-23T18:48:52.257395Z',
  'expiration_time': None,
  'self_trade_prevention_type': '',
  'action': 'buy',
  'side': 'no',
  'type': 'limit',
  'client_order_id': '5f82e4e5-ffa3-421a-a112-cd4b888cddb3',
  'order_group_id': ''}}